In [12]:
import json
from pathlib import Path

from openai import OpenAI
import pandas as pd
from pydantic import BaseModel

from tqdm import tqdm

In [8]:
llm = OpenAI()

In [9]:
files = list(Path('../data').rglob('*.md'))

In [10]:
with open("prompts/extraction.txt", "r") as f:
    prompt_extraction = f.read()

In [11]:
class NewsletterItem(BaseModel):
    headline: str
    url: str
    description: str

In [14]:
for nl in tqdm(files):
    with open(nl, "r") as f:
        data = f.read()

    structured = []
    chunks = data.split("* * *")

    for chunk in chunks:
        resp = llm.beta.chat.completions.parse(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": prompt_extraction
                },
                {
                    "role": "user",
                    "content": chunk
                }
            ],
            response_format=NewsletterItem
        )
        structured.append(resp.choices[0].message.content)
    
    structured = [json.loads(s) for s in structured]
    df = pd.DataFrame(structured)
    df.to_csv(nl.with_suffix(".csv"), index=False)

100%|██████████| 5/5 [01:37<00:00, 19.55s/it]
